In [ ]:
import os
import sys

# Add src to path to import infrastructure
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
# Updated import based on deprecation
try:
    from langchain.chains import LLMChain
except ImportError:
    # Fallback or alternative if available in environment
    pass

from infrastructure.client_requests import (
    make_request,
    consult_insurance_policy,
    report_emergency,
    consult_payments,
    schedule_inspection,
    manage_claims,
    quote_new_insurance,
    consult_bank_channel
)


In [2]:
from infrastructure.advanced_broker_vehicular import AdvancedBrokerVehicular

ImportError: cannot import name 'AdvancedBrokerVehicular' from 'infrastructure.advanced_broker_vehicular' (/Users/jessieblake/Desktop/Proyects/Underwriting_Insurance_app/underwriting_insurance/src/infrastructure/advanced_broker_vehicular.py)

# Chatbot Interface & Method Testing
this notebook tests the infrastructure methods for the insurance app.

In [ ]:
print("Testing: Consult Insurance Policy")
consult_insurance_policy()

In [ ]:
print("Testing: Report Emergency")
report_emergency()

In [ ]:
print("Testing: Consult Payments")
consult_payments()

In [ ]:
print("Testing: Schedule Inspection")
schedule_inspection()

In [ ]:
print("Testing: Manage Claims")
manage_claims()

In [ ]:
print("Testing: Quote New Insurance")
quote_new_insurance()

In [ ]:
print("Testing: Consult Bank Channel")
consult_bank_channel()

# Model Training
We will now generate synthetic data based on policy information rules and train an intent classification model.

In [ ]:
from infrastructure.rules import crear_dataset_rules
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Generate Synthetic Data
# Note: Ensure rules.py has been updated with all intents
df = crear_dataset_rules(n_pos_clas=1000)
print(f"Dataset generated with {len(df)} samples.")
print(df.head())

# 2. Split Data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# 3. Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier(random_state=42))
])

# 4. Train Model
print("Training model...")
pipeline.fit(X_train, y_train)
print("Model trained.")

# 5. Evaluate
print("Evaluating model...")
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


# Interactive Chatbot
Run the cell below to start a chat session with the bot. Type 'salir', 'exit', or 'quit' to end the session.

In [ ]:
def predict_intent(text):
    prediction = pipeline.predict([text])[0]
    return prediction

# Map intents to infrastructure functions
intent_actions = {
    "consultar_poliza": consult_insurance_policy,
    "reportar_emergencia": report_emergency,
    "pagos": consult_payments,
    "inspeccion_vehicular": schedule_inspection,
    "gestion_reclamos": manage_claims,
    "cotizar": quote_new_insurance,
    "consultar_banco": consult_bank_channel
}

print("🤖 Bot: Hola, soy tu asistente de seguros. ¿En qué puedo ayudarte hoy?")
print("(Escribe 'salir' para terminar)")

while True:
    try:
        user_input = input("\nTú: ")
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("🤖 Bot: ¡Hasta luego!")
            break
        
        if not user_input.strip():
            continue

        intent = predict_intent(user_input)
        print(f"[Debug] Intent detectado: {intent}")
        
        action = intent_actions.get(intent)
        if action:
            action()
        else:
            print("🤖 Bot: Lo siento, no entendí tu solicitud.")
            
    except KeyboardInterrupt:
        print("\n🤖 Bot: Interrupción detectada. Saliendo...")
        break
    except Exception as e:
        print(f"🤖 Bot: Ocurrió un error: {e}")
